In [1]:
import pandas as pd
import numpy as np
import os
import gzip
import io
from scripts.check_overlaps import check_overlaps, check_overlaps_and_annotate
from make_dataset import make_dataset_censored,  make_dataset_uncensored, repeat_masker, COSMIC_CGC_interactions, TF_binding_site_annotations, create_vep_input, read_vcf, long_range_interactions_results, create_vep_input, read_vcf, clean_and_preprocess

Rheinbay 2020 Supplementary Table 5<br>
We filter on the basis of 'Final filter judgement' = TRUE<br>
Then we take the unique element IDs<br>

In [13]:
df = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 5 Non-coding point mut dr')
# df1 = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 4 Protein-coding point mu')
df['file_name'] = df['ID'].apply(lambda x: x.split('::')[0]+'.bed')
df['file_name'] = df['file_name'].apply(lambda x: x.replace('mirna_', 'mirna.') if 'mirna_pre' in x or 'mirna_mat' in x else x)
noncoding_elements = list(df[df['Final filter judgement'] == True]['ID'].unique())
file_df = pd.read_csv('data/PCAWG_mutations_to_elements.icgc.public.txt', sep='\t')

In [14]:
rheinbay_noncoding_drivers = file_df[file_df['reg_id'].isin(noncoding_elements)][['mut_chr', 'mut_pos1', 'mut_pos2', 'mut_ref', 'mut_alt', 'region_type', 'reg_id']].drop_duplicates().reset_index(drop=True)
rheinbay_noncoding_drivers.rename({'mut_chr': 'chr', 'mut_pos1': 'start', 'mut_pos2': 'end', 'mut_ref': 'ref', 'mut_alt': 'alt'}, axis=1, inplace=True)
rheinbay_noncoding_drivers['data_source'] = 'Rheinbay et al 2020'
rheinbay_noncoding_drivers

,chr,start,end,ref,alt,region_type,reg_id,data_source
0,chr11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020
1,chr11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020
2,chr7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA,Rheinbay et al 2020
3,chr1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA,Rheinbay et al 2020
4,chr11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020
...,...,...,...,...,...,...,...,...
901,chr11,65194294,65194295,-,CATAATATTTTAAAA,lncrna.ncrna.bed,lncrna.ncrna::gencode::NEAT1::ENSG00000245532.4,Rheinbay et al 2020
902,chr11,65269272,65269273,AA,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::MALAT1::ENSG00000251562.3,Rheinbay et al 2020
903,chr11,65267659,65267659,A,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::MALAT1::ENSG00000251562.3,Rheinbay et al 2020
904,chr11,65209144,65209148,TTTAT,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::NEAT1::ENSG00000245532.4,Rheinbay et al 2020


In [15]:
dr_nod = pd.read_excel('data\Dr. Nod\gkac1251_supplemental_files\SupplementaryTable4.xlsx', engine='openpyxl')
dr_nod = dr_nod[['chr', 'pos0', 'pos1', 'ref', 'alt', 'tissue', 'geneSymbol']]
dr_nod.rename({'pos0': 'start', 'pos1': 'end', 'tissue': 'region_type', 'geneSymbol': 'reg_id'}, inplace=True, axis=1)
dr_nod['data_source'] = 'Dr.Nod 2023'
dr_nod

,chr,start,end,ref,alt,region_type,reg_id,data_source
0,chr1,19578253,19578254,G,A,Brain,MRTO4,Dr.Nod 2023
1,chr1,19578254,19578255,G,A,Brain,MRTO4,Dr.Nod 2023
2,chr1,23889291,23889292,G,T,Ovary,ID3,Dr.Nod 2023
3,chr1,23895459,23895460,G,T,Ovary,ID3,Dr.Nod 2023
4,chr1,23888768,23888769,T,C,Ovary,ID3,Dr.Nod 2023
...,...,...,...,...,...,...,...,...
210,chr9,125026995,125026996,C,G,Breast,MRRF,Dr.Nod 2023
211,chr9,124049461,124049462,T,A,Breast,MRRF,Dr.Nod 2023
212,chrX,70338403,70338404,G,A,Brain,MED12,Dr.Nod 2023
213,chrX,70401837,70401838,G,T,Brain,MED12,Dr.Nod 2023


In [23]:
test_data = pd.concat([rheinbay_noncoding_drivers, dr_nod])#[(rheinbay_noncoding_drivers['region_type'] == 'gc19_pc.promCore.bed') & (rheinbay_noncoding_drivers['reg_id'].str.contains('TERT'))]#.value_counts()
test_data.reset_index(drop=True, inplace=True)
test_data['id'] = 'test' + test_data.index.astype(str)
test_data['driver'] = 1
test_data.drop_duplicates(subset=['chr', 'start', 'end', 'ref', 'alt'], inplace=True)
test_data['chr'] = test_data['chr'].apply(lambda x: x.replace('chr', ''))
test_data

,chr,start,end,ref,alt,region_type,reg_id,data_source,id,driver
0,11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test0,1
1,11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test1,1
2,7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA,Rheinbay et al 2020,test2,1
3,1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA,Rheinbay et al 2020,test3,1
4,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA,Rheinbay et al 2020,test4,1
...,...,...,...,...,...,...,...,...,...,...
1116,9,125026995,125026996,C,G,Breast,MRRF,Dr.Nod 2023,test1116,1
1117,9,124049461,124049462,T,A,Breast,MRRF,Dr.Nod 2023,test1117,1
1118,X,70338403,70338404,G,A,Brain,MED12,Dr.Nod 2023,test1118,1
1119,X,70401837,70401838,G,T,Brain,MED12,Dr.Nod 2023,test1119,1


In [24]:
test_data.to_csv('./data/test_data_final.csv', index = False)

In [129]:
pd.read_pickle('data/repeat_masker.pickle')
# df = repeat_masker(df)
# df = COSMIC_CGC_interactions(df)
# df = TF_binding_site_annotations(df)
# df = long_range_interactions_results(df)
# df

,chr,start,end,allele,driver,chrom,itemsReturned,bin,swScore,milliDiv,...,genoEnd,genoLeft,strand,repName,repClass,repFamily,repStart,repEnd,repLeft,id
0,5,1295203,1295203,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
1,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
2,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
3,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
4,5,1295226,1295226,G/A,1,5,1.0,594.0,289.0,254.0,...,1295316.0,-179619944.0,+,(CGGGG)n,Simple_repeat,Simple_repeat,2.0,133.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,X,72254391,72254391,C/T,0,X,1.0,1136.0,27572.0,45.0,...,72257675.0,-83012885.0,-,L1PA3,LINE,L1,0.0,6155.0,1.0,4
213,3,180585989,180585989,C/T,0,3,1.0,1962.0,1597.0,174.0,...,180586453.0,-17435977.0,-,LTR1C,LTR,ERV1,0.0,656.0,1.0,2
214,19,30558759,30558759,T/C,0,19,1.0,818.0,1071.0,167.0,...,30559427.0,-28569556.0,+,(TCCA)n,Simple_repeat,Simple_repeat,1.0,706.0,0.0,1
215,16,62818734,62818734,C/T,0,16,1.0,1064.0,9249.0,235.0,...,62822231.0,-27532522.0,-,L1MB8,LINE,L1,-1.0,7037.0,3103.0,1


In [19]:
arr_ids = []
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/graylist'))
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/graylist'))
arr_ids = [s for s in arr_ids if '.tbi' not in s]
arr_ids_names = [s[:s.find(".")] for s in arr_ids]

In [ ]:
def read_vcf_gzip(filename):
    with gzip.open(filename, 'r') as f:
        lines = [l for l in f if not l.startswith(b'##')]
    df = pd.read_csv(
        io.StringIO(b''.join(lines).decode()),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return df
    
read_vcf_gzip('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel/0009b464-b376-4fbc-8a56-da538269a02f.consensus.20161006.somatic.indel.vcf.gz')

In [3]:
def row_count(input):
    with open(input) as f:
        for i, l in enumerate(f):
            pass
    return i

row_count('data/Rheinbay 2020/final_consensus_passonly.snv_mnv_indel.icgc.controlled.maf')

23159591